In [592]:
import pandas as pd
from matplotlib import pyplot as plt
import re
import os
if "R_HOME" not in os.environ:
    os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Resources/'
import numpy as np
import rpy2.robjects.numpy2ri
import rpy2.robjects as R
from rpy2.robjects.packages import importr
rpy2.robjects.numpy2ri.activate()
import statsmodels.stats.multitest as sm
import scipy.stats as ss

R.r('set.seed')(1)

R_STATS = importr('stats')

In [720]:
def fisher_exact_5x2(matrix, numiter=100000):
    return R_STATS.fisher_test(matrix, simulate_p_value=True, B=numiter)

def fisher_exact_2x2(matrix, alt):
    return R_STATS.fisher_test(matrix, alternative=alt)

In [242]:
# I accidentally named DLBCL_C_D_1142_NULLPAIR as DLBCL_C_D_1140_NULLPAIR. Manually fix for now
# The second DLBCL_C_D_1140_NULLPAIR (index 249) is named incorrectly

In [452]:
population_frequencies = pd.read_csv('../../data_tables/hla_typing/allelefrequencies.net.HLA.AF.US.Caucasians.20Feb2023.tsv',
                                    sep='\t', index_col=0)


In [453]:
labels = pd.read_csv('../../data_tables/confidence_tables/baseline_probabilities.connectivity_based.sensitivity_power2.Sep_23_2022.tsv',
                    sep='\t', index_col=0)
hla_types_staudt = pd.read_csv('../../data_tables/hla_typing/staudt_samples_optitype_aggregated.tsv', sep='\t', index_col=0)

hla_types_shipp = pd.read_csv('../../data_tables/hla_typing/shipp_samples_optitype_aggregated.tsv', sep='\t', index_col=0)
hla_types_shipp_normals = pd.read_csv('../../data_tables/hla_typing/shipp_samples_normals_optitype_aggregated.tsv', sep='\t', index_col=0)
hla_types_staudt = hla_types_staudt.loc[hla_types_staudt.index.isin(labels.index)]
filepath_re = re.compile('.*\/(.*)')
cleaned_index = []
for i in hla_types_shipp.index:
    sample = filepath_re.findall(i)[0]
    cleaned_index.append(sample)
    
cleaned_index[249] = 'DLBCL_C_D_1142_NULLPAIR'
hla_types_shipp.index = cleaned_index
hla_types_shipp = hla_types_shipp.loc[hla_types_shipp.index.isin(labels.index)]

cleaned_index = []
for i in hla_types_shipp_normals.index:
    sample = filepath_re.findall(i)[0]
    cleaned_index.append(sample)
    
hla_types_shipp_normals.index = cleaned_index
hla_types_shipp_normals = hla_types_shipp_normals.loc[hla_types_shipp_normals.index.isin(labels.index)]

hla_types = pd.concat([hla_types_shipp, hla_types_staudt])
hla_types

,A1,A2,B1,B2,C1,C2,Reads,Objective
DFCIDL008_DT,A*32:01,A*03:01,B*07:02,B*27:05,C*01:02,C*07:02,1668,1607.952
DFCIDL009_DT,A*02:01,A*11:01,B*13:02,B*35:01,C*06:02,C*04:01,1642,1568.110
DLBCL_DFCI_DLBCL_GOE05,A*02:01,A*29:01,B*35:01,B*57:01,C*06:02,C*04:01,1094,1054.616
DLBCL_DFCI_DLBCL_GOE07,A*01:01,A*03:01,B*08:01,B*07:02,C*07:02,C*07:01,1486,1419.130
DLBCL_DFCI_DLBCL_GOE16,A*02:01,A*01:01,B*08:01,B*08:01,C*07:01,C*07:01,1377,1339.821
...,...,...,...,...,...,...,...,...
DLBCL10536,A*01:01,A*03:01,B*07:02,B*35:03,C*07:02,C*04:01,3140,2998.700
DLBCL10971,A*02:01,A*02:01,B*15:01,B*38:01,C*03:03,C*12:03,2621,2550.233
DLBCL10887,A*02:01,A*02:01,B*15:01,B*15:01,C*03:03,C*03:04,2787,2761.917
DLBCL10507,A*29:02,A*24:03,B*44:03,B*35:08,C*04:01,C*16:01,2579,2462.945


In [454]:
for idx in hla_types_shipp_normals.index:
    print(idx)
    print(list(hla_types_shipp_normals.loc[idx]))
    print(list(hla_types_shipp.loc[idx]))
    print('---------------')

DLBCL_DFCI_DLBCL_GOE05
['A*29:01', 'A*02:01', 'B*57:01', 'B*35:01', 'C*06:02', 'C*04:01', 1245, 1200.1800000000023]
['A*02:01', 'A*29:01', 'B*35:01', 'B*57:01', 'C*06:02', 'C*04:01', 1094, 1054.6160000000025]
---------------
DLBCL_DFCI_DLBCL_GOE07
['A*01:01', 'A*03:01', 'B*08:01', 'B*07:02', 'C*07:01', 'C*07:02', 1485, 1418.1749999999995]
['A*01:01', 'A*03:01', 'B*08:01', 'B*07:02', 'C*07:02', 'C*07:01', 1486, 1419.13]
---------------
DLBCL_DFCI_DLBCL_GOE16
['A*02:01', 'A*01:01', 'B*08:01', 'B*08:01', 'C*07:01', 'C*07:01', 1149, 1117.9769999999994]
['A*02:01', 'A*01:01', 'B*08:01', 'B*08:01', 'C*07:01', 'C*07:01', 1377, 1339.8209999999988]
---------------
DLBCL_LS2325
['A*01:01', 'A*03:01', 'B*08:01', 'B*47:01', 'C*07:01', 'C*06:02', 1431, 1353.7260000000017]
['A*03:01', 'A*03:01', 'B*07:02', 'B*35:01', 'C*04:01', 'C*07:02', 1662, 1617.1259999999988]
---------------
DLBCL_LS4054
['A*30:02', 'A*11:01', 'B*51:01', 'B*27:05', 'C*02:02', 'C*14:02', 1632, 1558.5599999999986]
['A*30:02', 'A*

In [455]:
gsm.loc['X6Q21.DEL'].sort_values(ascending=False)[0:20]

DLBC_FF_8042_TP_NB         2.0
DLBCL10485                 2.0
DLBCL10512                 2.0
DLBCL_RICOVER_467          2.0
DLBCL_C_D_1166_NULLPAIR    2.0
DLBCL11476                 2.0
DLBCL_RICOVER_704          1.0
DLBCL_RICOVER_1060         1.0
DLBCL_RICOVER_583          1.0
DLBCL10880                 1.0
DLBCL11514                 1.0
DLBCL10909                 1.0
DLBCL10534                 1.0
DLBCL10963                 1.0
DLBCL_RICOVER_945          1.0
DLBCL_RICOVER_417          1.0
DLBCL11566                 1.0
DLBCL11206                 1.0
DLBCL10893                 1.0
DLBCL_MC_F210_GWC          1.0
Name: X6Q21.DEL, dtype: object

In [456]:
gsm = pd.read_csv('../../data_tables/gsm/DLBCL.699.fullGSM.Sep_23_2022.tsv', sep='\t', index_col=0)

In [457]:
gsm_staudt = gsm[hla_types_staudt.index]

In [458]:
gsm_staudt

,DLBCL11539,DLBCL10879,DLBCL11473,DLBCL10514,DLBCL10881,DLBCL11560,DLBCL11528,DLBCL11482,DLBCL10950,DLBCL10546,...,DLBCL11353,DLBCL10873,DLBCL10981,DLBCL10518,DLBCL11486,DLBCL10536,DLBCL10971,DLBCL10887,DLBCL10507,DLBCL11466
0,,,,,,,,,,,,,,,,,,,,,
STAT3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
STK33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OSBPL10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
BCL11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PIM1,0.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,2.0,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SV.BCL2.CCF,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SV.BCL6.CCF,1,0,1,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
SV.MYC.CCF,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0


In [459]:
hla_types_staudt

,A1,A2,B1,B2,C1,C2,Reads,Objective
DLBCL11539,A*31:01,A*01:01,B*40:01,B*40:01,C*03:04,C*03:04,3864,3759.672
DLBCL10879,A*01:01,A*11:01,B*08:01,B*55:01,C*03:03,C*07:01,3361,3179.506
DLBCL11473,A*01:01,A*03:01,B*08:01,B*51:01,C*07:01,C*14:02,2472,2338.512
DLBCL10514,A*02:01,A*11:01,B*35:01,B*51:01,C*02:02,C*04:01,4053,3870.605
DLBCL10881,A*01:01,A*25:01,B*44:02,B*08:01,C*05:01,C*07:01,2205,2105.775
...,...,...,...,...,...,...,...,...
DLBCL10536,A*01:01,A*03:01,B*07:02,B*35:03,C*07:02,C*04:01,3140,2998.700
DLBCL10971,A*02:01,A*02:01,B*15:01,B*38:01,C*03:03,C*12:03,2621,2550.233
DLBCL10887,A*02:01,A*02:01,B*15:01,B*15:01,C*03:03,C*03:04,2787,2761.917
DLBCL10507,A*29:02,A*24:03,B*44:03,B*35:08,C*04:01,C*16:01,2579,2462.945


In [460]:
hla_types['A1'].value_counts()

A*02:01    273
A*01:01    115
A*03:01     42
A*24:02     38
A*31:01     32
A*29:02     25
A*11:01     21
A*32:01     20
A*33:01     16
A*30:01     13
A*02:05     13
A*30:02     12
A*26:01     10
A*33:03      8
A*25:01      7
A*02:06      5
A*68:02      4
A*29:01      4
A*02:07      4
A*24:07      4
A*68:01      3
A*02:02      3
A*66:01      2
A*23:01      2
A*34:02      2
A*11:02      2
A*74:01      1
A*02:17      1
A*69:01      1
A*33:05      1
A*66:02      1
A*02:03      1
A*74:02      1
A*30:04      1
A*24:03      1
A*03:02      1
Name: A1, dtype: int64

In [461]:
c1_s = labels.loc[labels['cluster'] == 1].index
c2_s = labels.loc[labels['cluster'] == 2].index
c3_s = labels.loc[labels['cluster'] == 3].index
c4_s = labels.loc[labels['cluster'] == 4].index
c5_s = labels.loc[labels['cluster'] == 5].index

In [462]:
c1_hlas = hla_types.loc[hla_types.index.isin(c1_s)]
c2_hlas = hla_types.loc[hla_types.index.isin(c2_s)]
c3_hlas = hla_types.loc[hla_types.index.isin(c3_s)]
c4_hlas = hla_types.loc[hla_types.index.isin(c4_s)]
c5_hlas = hla_types.loc[hla_types.index.isin(c5_s)]

In [463]:
clus_hlas = [('Clus1', c1_hlas), ('Clus2', c2_hlas), ('Clus3', c3_hlas), ('Clus4', c4_hlas), ('Clus5', c5_hlas)]
clusters = ['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']
hla_counts_pairs = pd.DataFrame(columns=clusters)

for ele in clus_hlas:
    clus = ele[0]
    curr_hlas = ele[1]
    for s in curr_hlas.index:
        hlas_A = '_'.join(sorted([curr_hlas.loc[s, 'A1'], curr_hlas.loc[s, 'A2']]))
        hlas_B = '_'.join(sorted([curr_hlas.loc[s, 'B1'], curr_hlas.loc[s, 'B2']]))
        hlas_C = '_'.join(sorted([curr_hlas.loc[s, 'C1'], curr_hlas.loc[s, 'C2']]))
        
        if hlas_A not in hla_counts_pairs.index:
            hla_counts_pairs.loc[hlas_A] = 0
        if hlas_B not in hla_counts_pairs.index:
            hla_counts_pairs.loc[hlas_B] = 0
        if hlas_C not in hla_counts_pairs.index:
            hla_counts_pairs.loc[hlas_C] = 0
            
        hla_counts_pairs.loc[hlas_A, clus] += 1
        hla_counts_pairs.loc[hlas_B, clus] += 1
        hla_counts_pairs.loc[hlas_C, clus] += 1
        
hla_counts_pairs['total'] = hla_counts_pairs.sum(axis=1).astype(int)
hla_counts_pairs['max_delta'] = hla_counts_pairs.iloc[:, 0:5].max(axis=1) - hla_counts_pairs.iloc[:, 0:5].min(axis=1)
hla_counts_pairs = hla_counts_pairs.sort_values(by='max_delta', ascending=False)
hla_counts_pairs

,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,8,13,8,4,15,48,11.0
A*02:01_A*02:01,15,14,12,10,20,71,10.0
A*02:01_A*11:01,4,4,7,1,9,25,8.0
C*03:04_C*07:01,4,2,2,1,9,18,8.0
A*01:01_A*24:02,4,0,4,2,8,18,8.0
...,...,...,...,...,...,...,...
C*04:01_C*07:04,0,1,1,0,1,3,1.0
B*35:01_B*57:03,0,1,0,0,0,1,1.0
B*35:01_B*38:01,0,1,0,0,0,1,1.0
B*35:01_B*40:01,0,1,0,0,1,2,1.0


In [464]:
hla_counts_pairs_percent = hla_counts_pairs.copy(deep=True)
hla_counts_pairs_percent['Clus1'] = hla_counts_pairs_percent['Clus1'] / len(c1_s)
hla_counts_pairs_percent['Clus2'] = hla_counts_pairs_percent['Clus2'] / len(c2_s)
hla_counts_pairs_percent['Clus3'] = hla_counts_pairs_percent['Clus3'] / len(c3_s)
hla_counts_pairs_percent['Clus4'] = hla_counts_pairs_percent['Clus4'] / len(c4_s)
hla_counts_pairs_percent['Clus5'] = hla_counts_pairs_percent['Clus5'] / len(c5_s)

In [465]:
clus_hlas = [('Clus1', c1_hlas), ('Clus2', c2_hlas), ('Clus3', c3_hlas), ('Clus4', c4_hlas), ('Clus5', c5_hlas)]
clusters = ['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']
hla_alleles = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
hla_counts = pd.DataFrame(columns=[x + '_' + y for y in hla_alleles for x in clusters])

for ele in clus_hlas:
    clus = ele[0]
    curr_hlas = ele[1]
    for a in hla_alleles:
        counts = curr_hlas[a].value_counts()
        for c in counts.index:
            if c not in hla_counts.index:
                hla_counts.loc[c] = 0
            
            hla_counts.loc[c, clus + '_' + a] = counts[c]

hla_counts = hla_counts.sort_index()
hla_counts

,Clus1_A1,Clus2_A1,Clus3_A1,Clus4_A1,Clus5_A1,Clus1_A2,Clus2_A2,Clus3_A2,Clus4_A2,Clus5_A2,...,Clus1_C1,Clus2_C1,Clus3_C1,Clus4_C1,Clus5_C1,Clus1_C2,Clus2_C2,Clus3_C2,Clus4_C2,Clus5_C2
A*01:01,21,27,19,20,28,22,21,26,15,25,...,0,0,0,0,0,0,0,0,0,0
A*02:01,50,59,54,33,77,25,26,15,15,26,...,0,0,0,0,0,0,0,0,0,0
A*02:02,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A*02:03,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A*02:05,4,3,3,1,2,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C*16:01,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,9,5,3,3,1
C*16:02,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,2,1,3,2,1
C*16:04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
C*17:01,0,0,0,0,0,0,0,0,0,0,...,1,3,0,2,1,3,0,0,1,0


In [466]:
hla_counts_percent = hla_counts.copy(deep=True)
for c in hla_counts_percent.columns:
    if 'Clus1' in c:
        hla_counts_percent[c] = hla_counts_percent[c] / len(c1_s)
    elif 'Clus2' in c:
        hla_counts_percent[c] = hla_counts_percent[c] / len(c2_s)
    elif 'Clus3' in c:
        hla_counts_percent[c] = hla_counts_percent[c] / len(c3_s)
    elif 'Clus4' in c:
        hla_counts_percent[c] = hla_counts_percent[c] / len(c4_s)
    elif 'Clus5' in c:
        hla_counts_percent[c] = hla_counts_percent[c] / len(c5_s)

In [467]:
#hla_counts = hla_counts.astype(str) + ' (' + hla_counts_percent.astype(float).round(3).astype(str) + ')'

In [468]:
hla_counts_A = hla_counts.loc[hla_counts.index.str.contains('A\*'), hla_counts.columns.str.contains('_A')]
hla_counts_B = hla_counts.loc[hla_counts.index.str.contains('B\*'), hla_counts.columns.str.contains('_B')]
hla_counts_C = hla_counts.loc[hla_counts.index.str.contains('C\*'), hla_counts.columns.str.contains('_C')]


hla_counts_A_percent = hla_counts_percent.loc[hla_counts_percent.index.str.contains('A\*'), 
                                      hla_counts_percent.columns.str.contains('_A')]
hla_counts_B_percent = hla_counts_percent.loc[hla_counts_percent.index.str.contains('B\*'), 
                                      hla_counts_percent.columns.str.contains('_B')]
hla_counts_C_percent = hla_counts_percent.loc[hla_counts_percent.index.str.contains('C\*'), 
                                      hla_counts_percent.columns.str.contains('_C')]


In [469]:
hla_counts_pairs.insert(0, 'p', -1)

In [470]:
hla_counts_pairs['p'] = -1
hla_counts_pairs

,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,-1,8,13,8,4,15,48,11.0
A*02:01_A*02:01,-1,15,14,12,10,20,71,10.0
A*02:01_A*11:01,-1,4,4,7,1,9,25,8.0
C*03:04_C*07:01,-1,4,2,2,1,9,18,8.0
A*01:01_A*24:02,-1,4,0,4,2,8,18,8.0
...,...,...,...,...,...,...,...,...
C*04:01_C*07:04,-1,0,1,1,0,1,3,1.0
B*35:01_B*57:03,-1,0,1,0,0,0,1,1.0
B*35:01_B*38:01,-1,0,1,0,0,0,1,1.0
B*35:01_B*40:01,-1,0,1,0,0,1,2,1.0


In [471]:
hla_counts_pairs['p'] = -1
for idx, row in hla_counts_pairs.iterrows():
    curr_counts = row[['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']]
    m = np.array([[curr_counts['Clus1'], len(c1_s) - curr_counts['Clus1']],
                  [curr_counts['Clus2'], len(c2_s) - curr_counts['Clus2']],
                  [curr_counts['Clus3'], len(c3_s) - curr_counts['Clus3']],
                  [curr_counts['Clus4'], len(c4_s) - curr_counts['Clus4']],
                  [curr_counts['Clus5'], len(c5_s) - curr_counts['Clus5']]])
    
    p = fisher_exact_5x2(m)[0][0]
    hla_counts_pairs.loc[idx, 'p'] = p

hla_counts_pairs

,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,0.686093,8,13,8,4,15,48,11.0
A*02:01_A*02:01,0.886731,15,14,12,10,20,71,10.0
A*02:01_A*11:01,0.210078,4,4,7,1,9,25,8.0
C*03:04_C*07:01,0.173168,4,2,2,1,9,18,8.0
A*01:01_A*24:02,0.037730,4,0,4,2,8,18,8.0
...,...,...,...,...,...,...,...,...
C*04:01_C*07:04,0.926461,0,1,1,0,1,3,1.0
B*35:01_B*57:03,0.751932,0,1,0,0,0,1,1.0
B*35:01_B*38:01,0.753292,0,1,0,0,0,1,1.0
B*35:01_B*40:01,1.000000,0,1,0,0,1,2,1.0


In [472]:
hla_counts_pairs.insert(0, 'q', np.nan)
hla_counts_pairs

,q,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,NaN,0.686093,8,13,8,4,15,48,11.0
A*02:01_A*02:01,NaN,0.886731,15,14,12,10,20,71,10.0
A*02:01_A*11:01,NaN,0.210078,4,4,7,1,9,25,8.0
C*03:04_C*07:01,NaN,0.173168,4,2,2,1,9,18,8.0
A*01:01_A*24:02,NaN,0.037730,4,0,4,2,8,18,8.0
...,...,...,...,...,...,...,...,...,...
C*04:01_C*07:04,NaN,0.926461,0,1,1,0,1,3,1.0
B*35:01_B*57:03,NaN,0.751932,0,1,0,0,0,1,1.0
B*35:01_B*38:01,NaN,0.753292,0,1,0,0,0,1,1.0
B*35:01_B*40:01,NaN,1.000000,0,1,0,0,1,2,1.0


In [473]:
hla_counts_pairs.loc[hla_counts_pairs['total'] >= 10]

,q,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,NaN,0.686093,8,13,8,4,15,48,11.0
A*02:01_A*02:01,NaN,0.886731,15,14,12,10,20,71,10.0
A*02:01_A*11:01,NaN,0.210078,4,4,7,1,9,25,8.0
C*03:04_C*07:01,NaN,0.173168,4,2,2,1,9,18,8.0
A*01:01_A*24:02,NaN,0.037730,4,0,4,2,8,18,8.0
C*06:02_C*07:01,NaN,0.421796,3,11,5,3,6,28,8.0
A*01:01_A*02:01,NaN,0.777512,13,14,15,10,17,69,7.0
C*07:01_C*12:03,NaN,0.035110,7,4,0,0,4,15,7.0
C*06:02_C*07:02,NaN,0.003090,1,7,1,5,0,14,7.0
C*07:01_C*07:01,NaN,0.871871,4,7,4,3,9,27,6.0


In [474]:
valid_indices = hla_counts_pairs.loc[hla_counts_pairs['total'] >= 10].index
hla_counts_pairs.loc[valid_indices, 'q'] = sm.multipletests(hla_counts_pairs.loc[valid_indices, 'p'], 
                                                            method='fdr_bh')[1]
hla_counts_pairs

,q,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,0.938342,0.686093,8,13,8,4,15,48,11.0
A*02:01_A*02:01,0.938342,0.886731,15,14,12,10,20,71,10.0
A*02:01_A*11:01,0.938342,0.210078,4,4,7,1,9,25,8.0
C*03:04_C*07:01,0.938342,0.173168,4,2,2,1,9,18,8.0
A*01:01_A*24:02,0.465332,0.037730,4,0,4,2,8,18,8.0
...,...,...,...,...,...,...,...,...,...
C*04:01_C*07:04,NaN,0.926461,0,1,1,0,1,3,1.0
B*35:01_B*57:03,NaN,0.751932,0,1,0,0,0,1,1.0
B*35:01_B*38:01,NaN,0.753292,0,1,0,0,0,1,1.0
B*35:01_B*40:01,NaN,1.000000,0,1,0,0,1,2,1.0


In [475]:
hla_counts_pairs.loc[valid_indices]

,q,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
A*02:01_A*03:01,0.938342,0.686093,8,13,8,4,15,48,11.0
A*02:01_A*02:01,0.938342,0.886731,15,14,12,10,20,71,10.0
A*02:01_A*11:01,0.938342,0.210078,4,4,7,1,9,25,8.0
C*03:04_C*07:01,0.938342,0.173168,4,2,2,1,9,18,8.0
A*01:01_A*24:02,0.465332,0.037730,4,0,4,2,8,18,8.0
C*06:02_C*07:01,0.938342,0.421796,3,11,5,3,6,28,8.0
A*01:01_A*02:01,0.938342,0.777512,13,14,15,10,17,69,7.0
C*07:01_C*12:03,0.465332,0.035110,7,4,0,0,4,15,7.0
C*06:02_C*07:02,0.114329,0.003090,1,7,1,5,0,14,7.0
C*07:01_C*07:01,0.938342,0.871871,4,7,4,3,9,27,6.0


In [476]:
hla_counts_pairs.sort_values(by='q', ascending=True)

,q,p,Clus1,Clus2,Clus3,Clus4,Clus5,total,max_delta
C*06:02_C*07:02,0.114329,0.003090,1,7,1,5,0,14,7.0
A*01:01_A*24:02,0.465332,0.037730,4,0,4,2,8,18,8.0
C*07:01_C*12:03,0.465332,0.035110,7,4,0,0,4,15,7.0
C*03:04_C*07:02,0.724823,0.093769,1,1,2,5,4,13,4.0
B*35:01_B*44:02,0.724823,0.097949,1,6,3,0,1,11,6.0
...,...,...,...,...,...,...,...,...,...
C*04:01_C*07:04,NaN,0.926461,0,1,1,0,1,3,1.0
B*35:01_B*57:03,NaN,0.751932,0,1,0,0,0,1,1.0
B*35:01_B*38:01,NaN,0.753292,0,1,0,0,0,1,1.0
B*35:01_B*40:01,NaN,1.000000,0,1,0,0,1,2,1.0


In [632]:
# HLA counts combined analysis

In [665]:
cols = ['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']
hla_counts_A_combined = pd.DataFrame([hla_counts_A['Clus1_A1'] + hla_counts_A['Clus1_A2'],
                                      hla_counts_A['Clus2_A1'] + hla_counts_A['Clus2_A2'],
                                      hla_counts_A['Clus3_A1'] + hla_counts_A['Clus3_A2'],
                                      hla_counts_A['Clus4_A1'] + hla_counts_A['Clus4_A2'],
                                      hla_counts_A['Clus5_A1'] + hla_counts_A['Clus5_A2']]).T
hla_counts_A_combined.columns = cols
hla_counts_A_combined['total'] = hla_counts_A_combined.sum(axis=1)

hla_counts_B_combined = pd.DataFrame([hla_counts_B['Clus1_B1'] + hla_counts_B['Clus1_B2'],
                                      hla_counts_B['Clus2_B1'] + hla_counts_B['Clus2_B2'],
                                      hla_counts_B['Clus3_B1'] + hla_counts_B['Clus3_B2'],
                                      hla_counts_B['Clus4_B1'] + hla_counts_B['Clus4_B2'],
                                      hla_counts_B['Clus5_B1'] + hla_counts_B['Clus5_B2']]).T
hla_counts_B_combined.columns = cols
hla_counts_B_combined['total'] = hla_counts_B_combined.sum(axis=1)

hla_counts_C_combined = pd.DataFrame([hla_counts_C['Clus1_C1'] + hla_counts_C['Clus1_C2'],
                                      hla_counts_C['Clus2_C1'] + hla_counts_C['Clus2_C2'],
                                      hla_counts_C['Clus3_C1'] + hla_counts_C['Clus3_C2'],
                                      hla_counts_C['Clus4_C1'] + hla_counts_C['Clus4_C2'],
                                      hla_counts_C['Clus5_C1'] + hla_counts_C['Clus5_C2']]).T
hla_counts_C_combined.columns = cols
hla_counts_C_combined['total'] = hla_counts_C_combined.sum(axis=1)

hla_counts_A_combined.insert(0, 'cluster_fisher5x2_p', -1)
hla_counts_B_combined.insert(0, 'cluster_fisher5x2_p', -1)
hla_counts_C_combined.insert(0, 'cluster_fisher5x2_p', -1)

In [666]:
hla_counts_A_combined.head()

,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total
A*01:01,-1,43,48,45,35,53,224
A*02:01,-1,75,85,69,48,103,380
A*02:02,-1,1,1,0,1,0,3
A*02:03,-1,0,1,0,0,0,1
A*02:05,-1,5,4,4,1,2,16


In [667]:
hla_counts_B_combined.head()

,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total
B*07:02,-1,24,29,26,26,33,138
B*07:04,-1,0,1,0,1,0,2
B*07:05,-1,0,0,0,3,0,3
B*08:01,-1,37,43,29,30,46,185
B*08:23,-1,1,0,0,0,0,1


In [668]:
hla_counts_C_combined.head()

,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total
C*01:02,-1,7,20,12,6,5,50
C*01:03,-1,1,0,0,0,0,1
C*02:02,-1,9,8,11,6,18,52
C*02:10,-1,0,0,1,0,0,1
C*03:02,-1,2,0,1,3,1,7


In [669]:
for idx, row in hla_counts_A_combined.iterrows():
    curr_counts = row[['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']]
    m = np.array([[curr_counts['Clus1'], len(c1_s) - curr_counts['Clus1']],
                  [curr_counts['Clus2'], len(c2_s) - curr_counts['Clus2']],
                  [curr_counts['Clus3'], len(c3_s) - curr_counts['Clus3']],
                  [curr_counts['Clus4'], len(c4_s) - curr_counts['Clus4']],
                  [curr_counts['Clus5'], len(c5_s) - curr_counts['Clus5']]])
    
    p = fisher_exact_5x2(m)[0][0]
    hla_counts_A_combined.loc[idx, 'cluster_fisher5x2_p'] = p
    
for idx, row in hla_counts_B_combined.iterrows():
    curr_counts = row[['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']]
    m = np.array([[curr_counts['Clus1'], len(c1_s) - curr_counts['Clus1']],
                  [curr_counts['Clus2'], len(c2_s) - curr_counts['Clus2']],
                  [curr_counts['Clus3'], len(c3_s) - curr_counts['Clus3']],
                  [curr_counts['Clus4'], len(c4_s) - curr_counts['Clus4']],
                  [curr_counts['Clus5'], len(c5_s) - curr_counts['Clus5']]])
    
    p = fisher_exact_5x2(m)[0][0]
    hla_counts_B_combined.loc[idx, 'cluster_fisher5x2_p'] = p
    
for idx, row in hla_counts_C_combined.iterrows():
    curr_counts = row[['Clus1', 'Clus2', 'Clus3', 'Clus4', 'Clus5']]
    m = np.array([[curr_counts['Clus1'], len(c1_s) - curr_counts['Clus1']],
                  [curr_counts['Clus2'], len(c2_s) - curr_counts['Clus2']],
                  [curr_counts['Clus3'], len(c3_s) - curr_counts['Clus3']],
                  [curr_counts['Clus4'], len(c4_s) - curr_counts['Clus4']],
                  [curr_counts['Clus5'], len(c5_s) - curr_counts['Clus5']]])
    
    p = fisher_exact_5x2(m)[0][0]
    hla_counts_C_combined.loc[idx, 'cluster_fisher5x2_p'] = p

In [670]:
valid_A = hla_counts_A_combined.loc[hla_counts_A_combined['total'] >= 10].index
valid_B = hla_counts_B_combined.loc[hla_counts_B_combined['total'] >= 10].index
valid_C = hla_counts_C_combined.loc[hla_counts_C_combined['total'] >= 10].index

hla_counts_A_combined.insert(0, 'cluster_q', np.nan)
hla_counts_B_combined.insert(0, 'cluster_q', np.nan)
hla_counts_C_combined.insert(0, 'cluster_q', np.nan)

hla_counts_A_combined.loc[valid_A, 'cluster_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'cluster_fisher5x2_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'cluster_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'cluster_fisher5x2_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'cluster_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'cluster_fisher5x2_p'], method='fdr_bh')[1]

hla_counts_A_combined

,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total
A*01:01,0.564082,0.344717,43,48,45,35,53,224
A*02:01,0.551814,0.275907,75,85,69,48,103,380
A*02:02,NaN,0.583764,1,1,0,1,0,3
A*02:03,NaN,0.751602,0,1,0,0,0,1
A*02:05,0.606864,0.501345,5,4,4,1,2,16
A*02:06,NaN,0.161258,2,2,3,0,0,7
A*02:07,NaN,0.348887,2,1,0,1,0,4
A*02:17,NaN,1.000000,0,0,0,0,1,1
A*02:74,NaN,1.000000,0,0,0,0,1,1
A*03:01,0.551814,0.194788,35,50,38,22,37,182


In [671]:
hla_counts_A_combined['pop_allele_f'] = np.nan
hla_counts_B_combined['pop_allele_f'] = np.nan
hla_counts_C_combined['pop_allele_f'] = np.nan
hla_counts_A_combined['pop_n'] = np.nan
hla_counts_B_combined['pop_n'] = np.nan
hla_counts_C_combined['pop_n'] = np.nan

In [672]:
a_als = hla_counts_A_combined.index[(hla_counts_A_combined.index.isin(population_frequencies.index))]
hla_counts_A_combined.loc[a_als, 'pop_allele_f'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_A_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Allele Frequency']
hla_counts_A_combined.loc[a_als, 'pop_n'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_A_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Sample Size']

b_als = hla_counts_B_combined.index[(hla_counts_B_combined.index.isin(population_frequencies.index))]
hla_counts_B_combined.loc[b_als, 'pop_allele_f'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_B_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Allele Frequency']
hla_counts_B_combined.loc[b_als, 'pop_n'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_B_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Sample Size']

c_als = hla_counts_C_combined.index[(hla_counts_C_combined.index.isin(population_frequencies.index))]
hla_counts_C_combined.loc[c_als, 'pop_allele_f'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_C_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Allele Frequency']
hla_counts_C_combined.loc[c_als, 'pop_n'] = population_frequencies.loc[(population_frequencies.index.isin(hla_counts_C_combined.index)) &
                                                                              (population_frequencies['Population'] == 'USA NMDP European Caucasian'),
                                                                             'Sample Size']


In [673]:
hla_counts_A_combined['allele_f_Clus1'] = hla_counts_A_combined['Clus1'] / hla_counts_A_combined['Clus1'].sum()
hla_counts_A_combined['allele_f_Clus2'] = hla_counts_A_combined['Clus2'] / hla_counts_A_combined['Clus2'].sum()
hla_counts_A_combined['allele_f_Clus3'] = hla_counts_A_combined['Clus3'] / hla_counts_A_combined['Clus3'].sum()
hla_counts_A_combined['allele_f_Clus4'] = hla_counts_A_combined['Clus4'] / hla_counts_A_combined['Clus4'].sum()
hla_counts_A_combined['allele_f_Clus5'] = hla_counts_A_combined['Clus5'] / hla_counts_A_combined['Clus5'].sum()
hla_counts_A_combined['allele_f_total'] = hla_counts_A_combined['total'] / hla_counts_A_combined['total'].sum()

hla_counts_B_combined['allele_f_Clus1'] = hla_counts_B_combined['Clus1'] / hla_counts_B_combined['Clus1'].sum()
hla_counts_B_combined['allele_f_Clus2'] = hla_counts_B_combined['Clus2'] / hla_counts_A_combined['Clus2'].sum()
hla_counts_B_combined['allele_f_Clus3'] = hla_counts_B_combined['Clus3'] / hla_counts_B_combined['Clus3'].sum()
hla_counts_B_combined['allele_f_Clus4'] = hla_counts_B_combined['Clus4'] / hla_counts_B_combined['Clus4'].sum()
hla_counts_B_combined['allele_f_Clus5'] = hla_counts_B_combined['Clus5'] / hla_counts_B_combined['Clus5'].sum()
hla_counts_B_combined['allele_f_total'] = hla_counts_B_combined['total'] / hla_counts_B_combined['total'].sum()

hla_counts_C_combined['allele_f_Clus1'] = hla_counts_C_combined['Clus1'] / hla_counts_C_combined['Clus1'].sum()
hla_counts_C_combined['allele_f_Clus2'] = hla_counts_C_combined['Clus2'] / hla_counts_C_combined['Clus2'].sum()
hla_counts_C_combined['allele_f_Clus3'] = hla_counts_C_combined['Clus3'] / hla_counts_C_combined['Clus3'].sum()
hla_counts_C_combined['allele_f_Clus4'] = hla_counts_C_combined['Clus4'] / hla_counts_C_combined['Clus4'].sum()
hla_counts_C_combined['allele_f_Clus5'] = hla_counts_C_combined['Clus5'] / hla_counts_C_combined['Clus5'].sum()
hla_counts_C_combined['allele_f_total'] = hla_counts_C_combined['total'] / hla_counts_C_combined['total'].sum()

In [674]:
hla_counts_A_combined.head()

,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
A*01:01,0.564082,0.344717,43,48,45,35,53,224,0.1646,"1,242,890",0.154676,0.141176,0.193966,0.182292,0.156805,0.162319
A*02:01,0.551814,0.275907,75,85,69,48,103,380,0.2755,"1,242,890",0.269784,0.250000,0.297414,0.250000,0.304734,0.275362
A*02:02,NaN,0.583764,1,1,0,1,0,3,0.0009,"1,242,890",0.003597,0.002941,0.000000,0.005208,0.000000,0.002174
A*02:03,NaN,0.751602,0,1,0,0,0,1,0.0000,"1,242,890",0.000000,0.002941,0.000000,0.000000,0.000000,0.000725
A*02:05,0.606864,0.501345,5,4,4,1,2,16,0.0097,"1,242,890",0.017986,0.011765,0.017241,0.005208,0.005917,0.011594


In [675]:
hla_counts_B_combined.head()

,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
B*07:02,0.647470,0.296757,24,29,26,26,33,138,0.1306,"1,242,890",0.086331,0.085294,0.112069,0.135417,0.097633,0.100000
B*07:04,NaN,0.442806,0,1,0,1,0,2,0.0003,"1,242,890",0.000000,0.002941,0.000000,0.005208,0.000000,0.001449
B*07:05,NaN,0.002810,0,0,0,3,0,3,0.0030,"1,242,890",0.000000,0.000000,0.000000,0.015625,0.000000,0.002174
B*08:01,0.937791,0.859641,37,43,29,30,46,185,0.1144,"1,242,890",0.133094,0.126471,0.125000,0.156250,0.136095,0.134058
B*08:23,NaN,0.506815,1,0,0,0,0,1,0.0000,"1,242,890",0.003597,0.000000,0.000000,0.000000,0.000000,0.000725


In [676]:
hla_counts_C_combined.head()

,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,Clus3,Clus4,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
C*01:02,0.155248,0.009810,7,20,12,6,5,50,0.0341,"1,242,890",0.025180,0.058824,0.051724,0.031250,0.014793,0.036232
C*01:03,NaN,0.506905,1,0,0,0,0,1,0.0000,"1,242,890",0.003597,0.000000,0.000000,0.000000,0.000000,0.000725
C*02:02,0.434936,0.280397,9,8,11,6,18,52,0.0435,"1,242,890",0.032374,0.023529,0.047414,0.031250,0.053254,0.037681
C*02:10,NaN,0.305497,0,0,1,0,0,1,NaN,NaN,0.000000,0.000000,0.004310,0.000000,0.000000,0.000725
C*03:02,NaN,0.114629,2,0,1,3,1,7,0.0022,"1,242,890",0.007194,0.000000,0.004310,0.015625,0.002959,0.005072


In [677]:
hla_counts_A_combined.insert(0, 'population_total_p', np.nan)
hla_counts_A_combined.insert(0, 'population_C5_p', np.nan)
hla_counts_A_combined.insert(0, 'population_C4_p', np.nan)
hla_counts_A_combined.insert(0, 'population_C3_p', np.nan)
hla_counts_A_combined.insert(0, 'population_C2_p', np.nan)
hla_counts_A_combined.insert(0, 'population_C1_p', np.nan)

hla_counts_B_combined.insert(0, 'population_total_p', np.nan)
hla_counts_B_combined.insert(0, 'population_C5_p', np.nan)
hla_counts_B_combined.insert(0, 'population_C4_p', np.nan)
hla_counts_B_combined.insert(0, 'population_C3_p', np.nan)
hla_counts_B_combined.insert(0, 'population_C2_p', np.nan)
hla_counts_B_combined.insert(0, 'population_C1_p', np.nan)

hla_counts_C_combined.insert(0, 'population_total_p', np.nan)
hla_counts_C_combined.insert(0, 'population_C5_p', np.nan)
hla_counts_C_combined.insert(0, 'population_C4_p', np.nan)
hla_counts_C_combined.insert(0, 'population_C3_p', np.nan)
hla_counts_C_combined.insert(0, 'population_C2_p', np.nan)
hla_counts_C_combined.insert(0, 'population_C1_p', np.nan)

In [678]:
hla_counts_A_combined.head()

,population_C1_p,population_C2_p,population_C3_p,population_C4_p,population_C5_p,population_total_p,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,...,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
A*01:01,NaN,NaN,NaN,NaN,NaN,NaN,0.564082,0.344717,43,48,...,53,224,0.1646,"1,242,890",0.154676,0.141176,0.193966,0.182292,0.156805,0.162319
A*02:01,NaN,NaN,NaN,NaN,NaN,NaN,0.551814,0.275907,75,85,...,103,380,0.2755,"1,242,890",0.269784,0.250000,0.297414,0.250000,0.304734,0.275362
A*02:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.583764,1,1,...,0,3,0.0009,"1,242,890",0.003597,0.002941,0.000000,0.005208,0.000000,0.002174
A*02:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.751602,0,1,...,0,1,0.0000,"1,242,890",0.000000,0.002941,0.000000,0.000000,0.000000,0.000725
A*02:05,NaN,NaN,NaN,NaN,NaN,NaN,0.606864,0.501345,5,4,...,2,16,0.0097,"1,242,890",0.017986,0.011765,0.017241,0.005208,0.005917,0.011594


In [679]:
hla_counts_A_combined['Clus1'].sum()

278

In [680]:
# p = scipy.stats.binomtest(AC, n=2*699, p=AF, alternative='two-sided')

c1_n = hla_counts_A_combined['Clus1'].sum()
c2_n = hla_counts_A_combined['Clus2'].sum()
c3_n = hla_counts_A_combined['Clus3'].sum()
c4_n = hla_counts_A_combined['Clus4'].sum()
c5_n = hla_counts_A_combined['Clus5'].sum()
total_n = hla_counts_A_combined['total'].sum()
for idx, row in hla_counts_A_combined.iterrows():
    c1_ac = row['Clus1']
    c2_ac = row['Clus2']
    c3_ac = row['Clus3']
    c4_ac = row['Clus4']
    c5_ac = row['Clus5']
    tot_ac = row['total']
    pop_af = row['pop_allele_f']
    
    if np.isnan(pop_af) or np.isnan(row['cluster_q']):
        continue
    
    p_c1 = ss.binomtest(c1_ac, n=c1_n, p=pop_af, alternative='two-sided').pvalue
    p_c2 = ss.binomtest(c2_ac, n=c2_n, p=pop_af, alternative='two-sided').pvalue
    p_c3 = ss.binomtest(c3_ac, n=c3_n, p=pop_af, alternative='two-sided').pvalue
    p_c4 = ss.binomtest(c4_ac, n=c4_n, p=pop_af, alternative='two-sided').pvalue
    p_c5 = ss.binomtest(c5_ac, n=c5_n, p=pop_af, alternative='two-sided').pvalue
    p_t = ss.binomtest(tot_ac, n=total_n, p=pop_af, alternative='two-sided').pvalue
    
    hla_counts_A_combined.loc[idx, 'population_C1_p'] = p_c1
    hla_counts_A_combined.loc[idx, 'population_C2_p'] = p_c2
    hla_counts_A_combined.loc[idx, 'population_C3_p'] = p_c3
    hla_counts_A_combined.loc[idx, 'population_C4_p'] = p_c4
    hla_counts_A_combined.loc[idx, 'population_C5_p'] = p_c5
    hla_counts_A_combined.loc[idx, 'population_total_p'] = p_t

In [681]:
# p = scipy.stats.binomtest(AC, n=2*699, p=AF, alternative='two-sided')

c1_n = hla_counts_B_combined['Clus1'].sum()
c2_n = hla_counts_B_combined['Clus2'].sum()
c3_n = hla_counts_B_combined['Clus3'].sum()
c4_n = hla_counts_B_combined['Clus4'].sum()
c5_n = hla_counts_B_combined['Clus5'].sum()
total_n = hla_counts_B_combined['total'].sum()
for idx, row in hla_counts_B_combined.iterrows():
    c1_ac = row['Clus1']
    c2_ac = row['Clus2']
    c3_ac = row['Clus3']
    c4_ac = row['Clus4']
    c5_ac = row['Clus5']
    tot_ac = row['total']
    pop_af = row['pop_allele_f']
    
    if np.isnan(pop_af) or np.isnan(row['cluster_q']):
        continue
    
    p_c1 = ss.binomtest(c1_ac, n=c1_n, p=pop_af, alternative='two-sided').pvalue
    p_c2 = ss.binomtest(c2_ac, n=c2_n, p=pop_af, alternative='two-sided').pvalue
    p_c3 = ss.binomtest(c3_ac, n=c3_n, p=pop_af, alternative='two-sided').pvalue
    p_c4 = ss.binomtest(c4_ac, n=c4_n, p=pop_af, alternative='two-sided').pvalue
    p_c5 = ss.binomtest(c5_ac, n=c5_n, p=pop_af, alternative='two-sided').pvalue
    p_t = ss.binomtest(tot_ac, n=total_n, p=pop_af, alternative='two-sided').pvalue
    
    hla_counts_B_combined.loc[idx, 'population_C1_p'] = p_c1
    hla_counts_B_combined.loc[idx, 'population_C2_p'] = p_c2
    hla_counts_B_combined.loc[idx, 'population_C3_p'] = p_c3
    hla_counts_B_combined.loc[idx, 'population_C4_p'] = p_c4
    hla_counts_B_combined.loc[idx, 'population_C5_p'] = p_c5
    hla_counts_B_combined.loc[idx, 'population_total_p'] = p_t

In [682]:
# p = scipy.stats.binomtest(AC, n=2*699, p=AF, alternative='two-sided')

c1_n = hla_counts_C_combined['Clus1'].sum()
c2_n = hla_counts_C_combined['Clus2'].sum()
c3_n = hla_counts_C_combined['Clus3'].sum()
c4_n = hla_counts_C_combined['Clus4'].sum()
c5_n = hla_counts_C_combined['Clus5'].sum()
total_n = hla_counts_C_combined['total'].sum()
for idx, row in hla_counts_C_combined.iterrows():
    c1_ac = row['Clus1']
    c2_ac = row['Clus2']
    c3_ac = row['Clus3']
    c4_ac = row['Clus4']
    c5_ac = row['Clus5']
    tot_ac = row['total']
    pop_af = row['pop_allele_f']
    
    if np.isnan(pop_af) or np.isnan(row['cluster_q']):
        continue
    
    p_c1 = ss.binomtest(c1_ac, n=c1_n, p=pop_af, alternative='two-sided').pvalue
    p_c2 = ss.binomtest(c2_ac, n=c2_n, p=pop_af, alternative='two-sided').pvalue
    p_c3 = ss.binomtest(c3_ac, n=c3_n, p=pop_af, alternative='two-sided').pvalue
    p_c4 = ss.binomtest(c4_ac, n=c4_n, p=pop_af, alternative='two-sided').pvalue
    p_c5 = ss.binomtest(c5_ac, n=c5_n, p=pop_af, alternative='two-sided').pvalue
    p_t = ss.binomtest(tot_ac, n=total_n, p=pop_af, alternative='two-sided').pvalue
    
    hla_counts_C_combined.loc[idx, 'population_C1_p'] = p_c1
    hla_counts_C_combined.loc[idx, 'population_C2_p'] = p_c2
    hla_counts_C_combined.loc[idx, 'population_C3_p'] = p_c3
    hla_counts_C_combined.loc[idx, 'population_C4_p'] = p_c4
    hla_counts_C_combined.loc[idx, 'population_C5_p'] = p_c5
    hla_counts_C_combined.loc[idx, 'population_total_p'] = p_t

In [683]:
hla_counts_A_combined.head()

,population_C1_p,population_C2_p,population_C3_p,population_C4_p,population_C5_p,population_total_p,cluster_q,cluster_fisher5x2_p,Clus1,Clus2,...,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
A*01:01,0.746169,0.272497,0.249069,0.496032,0.769199,0.855980,0.564082,0.344717,43,48,...,53,224,0.1646,"1,242,890",0.154676,0.141176,0.193966,0.182292,0.156805,0.162319
A*02:01,0.893226,0.302818,0.462641,0.467647,0.247250,1.000000,0.551814,0.275907,75,85,...,103,380,0.2755,"1,242,890",0.269784,0.250000,0.297414,0.250000,0.304734,0.275362
A*02:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.583764,1,1,...,0,3,0.0009,"1,242,890",0.003597,0.002941,0.000000,0.005208,0.000000,0.002174
A*02:03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.751602,0,1,...,0,1,0.0000,"1,242,890",0.000000,0.002941,0.000000,0.000000,0.000000,0.000725
A*02:05,0.202192,0.577019,0.294118,1.000000,0.777710,0.489489,0.606864,0.501345,5,4,...,2,16,0.0097,"1,242,890",0.017986,0.011765,0.017241,0.005208,0.005917,0.011594


In [684]:
valid_A = hla_counts_A_combined.loc[(~hla_counts_A_combined['population_C1_p'].isna()) &
                                    (~hla_counts_A_combined['cluster_q'].isna())].index
valid_B = hla_counts_B_combined.loc[(~hla_counts_B_combined['population_C1_p'].isna()) &
                                    (~hla_counts_B_combined['cluster_q'].isna())].index
valid_C = hla_counts_C_combined.loc[(~hla_counts_C_combined['population_C1_p'].isna()) &
                                    (~hla_counts_C_combined['cluster_q'].isna())].index

In [685]:
hla_counts_A_combined.insert(0, 'population_C5_q', np.nan)
hla_counts_A_combined.insert(0, 'population_C4_q', np.nan)
hla_counts_A_combined.insert(0, 'population_C3_q', np.nan)
hla_counts_A_combined.insert(0, 'population_C2_q', np.nan)
hla_counts_A_combined.insert(0, 'population_C1_q', np.nan)
hla_counts_A_combined.insert(0, 'population_total_q', np.nan)

hla_counts_B_combined.insert(0, 'population_C5_q', np.nan)
hla_counts_B_combined.insert(0, 'population_C4_q', np.nan)
hla_counts_B_combined.insert(0, 'population_C3_q', np.nan)
hla_counts_B_combined.insert(0, 'population_C2_q', np.nan)
hla_counts_B_combined.insert(0, 'population_C1_q', np.nan)
hla_counts_B_combined.insert(0, 'population_total_q', np.nan)

hla_counts_C_combined.insert(0, 'population_C5_q', np.nan)
hla_counts_C_combined.insert(0, 'population_C4_q', np.nan)
hla_counts_C_combined.insert(0, 'population_C3_q', np.nan)
hla_counts_C_combined.insert(0, 'population_C2_q', np.nan)
hla_counts_C_combined.insert(0, 'population_C1_q', np.nan)
hla_counts_C_combined.insert(0, 'population_total_q', np.nan)

In [686]:
hla_counts_A_combined.loc[valid_A, 'population_C1_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_C1_p'], method='fdr_bh')[1]
hla_counts_A_combined.loc[valid_A, 'population_C2_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_C2_p'], method='fdr_bh')[1]
hla_counts_A_combined.loc[valid_A, 'population_C3_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_C3_p'], method='fdr_bh')[1]
hla_counts_A_combined.loc[valid_A, 'population_C4_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_C4_p'], method='fdr_bh')[1]
hla_counts_A_combined.loc[valid_A, 'population_C5_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_C5_p'], method='fdr_bh')[1]
hla_counts_A_combined.loc[valid_A, 'population_total_q'] = sm.multipletests(hla_counts_A_combined.loc[valid_A,'population_total_p'], method='fdr_bh')[1]

hla_counts_B_combined.loc[valid_B, 'population_C1_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_C1_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'population_C2_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_C2_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'population_C3_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_C3_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'population_C4_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_C4_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'population_C5_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_C5_p'], method='fdr_bh')[1]
hla_counts_B_combined.loc[valid_B, 'population_total_q'] = sm.multipletests(hla_counts_B_combined.loc[valid_B,'population_total_p'], method='fdr_bh')[1]

hla_counts_C_combined.loc[valid_C, 'population_C1_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_C1_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'population_C2_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_C2_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'population_C3_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_C3_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'population_C4_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_C4_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'population_C5_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_C5_p'], method='fdr_bh')[1]
hla_counts_C_combined.loc[valid_C, 'population_total_q'] = sm.multipletests(hla_counts_C_combined.loc[valid_C,'population_total_p'], method='fdr_bh')[1]


In [687]:
hla_counts_pairs = hla_counts_pairs.sort_values(by='q', ascending=True)

In [688]:
hla_counts_A_combined = hla_counts_A_combined.sort_values(by='population_total_q', ascending=True)
hla_counts_B_combined = hla_counts_B_combined.sort_values(by='population_total_q', ascending=True)
hla_counts_C_combined = hla_counts_C_combined.sort_values(by='population_total_q', ascending=True)

In [689]:
hla_counts_A_combined

,population_total_q,population_C1_q,population_C2_q,population_C3_q,population_C4_q,population_C5_q,population_C1_p,population_C2_p,population_C3_p,population_C4_p,...,Clus5,total,pop_allele_f,pop_n,allele_f_Clus1,allele_f_Clus2,allele_f_Clus3,allele_f_Clus4,allele_f_Clus5,allele_f_total
A*33:03,0.003640,0.840842,0.615568,0.235426,0.007591,0.662121,0.223649,0.296626,0.039238,0.000422,...,2,14,0.0032,"1,242,890",0.007194,0.005882,0.012931,0.026042,0.005917,0.010145
A*29:02,0.003938,0.895403,0.832287,0.325117,0.744047,0.017356,0.743596,0.462382,0.072248,0.428892,...,2,26,0.0353,"1,242,890",0.028777,0.026471,0.012931,0.020833,0.005917,0.018841
A*33:01,0.038266,0.146132,0.615568,0.435567,0.831198,0.874923,0.008118,0.207419,0.120991,0.671032,...,3,21,0.0081,"1,242,890",0.025180,0.014706,0.017241,0.010417,0.008876,0.015217
A*32:01,0.088011,0.840842,1.000000,0.070869,0.658585,0.922043,0.256206,1.000000,0.003937,0.170153,...,11,33,0.0355,"1,242,890",0.021583,0.035294,0.004310,0.015625,0.032544,0.023913
A*68:02,0.089704,0.281518,0.471432,1.000000,0.658585,0.874923,0.031280,0.026191,1.000000,0.219528,...,3,20,0.0084,"1,242,890",0.021583,0.020588,0.004310,0.015625,0.008876,0.014493
A*11:01,0.345308,0.900591,0.615568,1.000000,0.699578,0.524423,0.900591,0.307784,0.890732,0.291258,...,28,98,0.0609,"1,242,890",0.061151,0.073529,0.056034,0.078125,0.082840,0.071014
A*30:01,0.622668,0.895403,0.969043,1.000000,0.699578,0.676608,0.594171,0.807535,1.000000,0.321935,...,2,14,0.0130,"1,242,890",0.007194,0.008824,0.012931,0.020833,0.005917,0.010145
A*68:01,0.622668,0.900591,1.000000,0.235426,0.658585,0.585491,0.864643,1.000000,0.038239,0.216549,...,6,36,0.0319,"1,242,890",0.032374,0.029412,0.008621,0.046875,0.017751,0.026087
A*26:01,0.622668,0.895403,0.903543,0.555146,0.885016,0.524423,0.600389,0.635892,0.337898,0.835848,...,16,48,0.0309,"1,242,890",0.035971,0.035294,0.017241,0.031250,0.047337,0.034783
A*24:02,0.622668,0.840842,0.568655,0.768190,0.831198,0.922043,0.280281,0.063184,0.554804,0.604241,...,29,106,0.0846,"1,242,890",0.064748,0.055882,0.094828,0.093750,0.085799,0.076812


In [690]:
hla_counts_pairs_percent.to_csv('../../data_tables/hla_typing/hla_counts_pairs_percent.tsv', sep='\t')
hla_counts_pairs.to_csv('../../data_tables/hla_typing/hla_counts_pairs.tsv', sep='\t')

hla_counts.to_csv('../../data_tables/hla_typing/hla_types.tsv', sep='\t')
hla_counts_A_combined.to_csv('../../data_tables/hla_typing/hla_A.tsv', sep='\t')
hla_counts_B_combined.to_csv('../../data_tables/hla_typing/hla_B.tsv', sep='\t')
hla_counts_C_combined.to_csv('../../data_tables/hla_typing/hla_C.tsv', sep='\t')

hla_counts_percent.to_csv('../../data_tables/hla_typing/hla_types_percent.tsv', sep='\t')
hla_counts_A_percent.to_csv('../../data_tables/hla_typing/hla_A_percent.tsv', sep='\t')
hla_counts_B_percent.to_csv('../../data_tables/hla_typing/hla_B_percent.tsv', sep='\t')
hla_counts_C_percent.to_csv('../../data_tables/hla_typing/hla_C_percent.tsv', sep='\t')

In [695]:
gsm = pd.read_csv('../../data_tables/gsm/DLBCL.699.163drivers.Sep_23_2022.tsv', sep='\t', index_col=0)
gsm = gsm.loc[['HLA.A', 'HLA.B', 'HLA.C']].drop('classifier_name', axis=1).T
gsm

gene,HLA.A,HLA.B,HLA.C
DLBCL11470,0,0,0
DLBCL10900,2,2,2
DLBC_FF_A7CQ_TP_NB,0,0,0
DLBCL10462,0,0,0
DLBCL_RICOVER_1081,0,0,0
...,...,...,...
DLBCL11515,0,0,0
DLBCL10491,0,0,0
DLBCL_RICOVER_1046,0,0,0
DLBCL10547,0,0,0


In [697]:
mutated_a = gsm.loc[gsm['HLA.A'] == 2].index
mutated_b = gsm.loc[gsm['HLA.B'] == 2].index
mutated_c = gsm.loc[gsm['HLA.C'] == 2].index

In [694]:
hla_types

,A1,A2,B1,B2,C1,C2,Reads,Objective
DFCIDL008_DT,A*32:01,A*03:01,B*07:02,B*27:05,C*01:02,C*07:02,1668,1607.952
DFCIDL009_DT,A*02:01,A*11:01,B*13:02,B*35:01,C*06:02,C*04:01,1642,1568.110
DLBCL_DFCI_DLBCL_GOE05,A*02:01,A*29:01,B*35:01,B*57:01,C*06:02,C*04:01,1094,1054.616
DLBCL_DFCI_DLBCL_GOE07,A*01:01,A*03:01,B*08:01,B*07:02,C*07:02,C*07:01,1486,1419.130
DLBCL_DFCI_DLBCL_GOE16,A*02:01,A*01:01,B*08:01,B*08:01,C*07:01,C*07:01,1377,1339.821
...,...,...,...,...,...,...,...,...
DLBCL10536,A*01:01,A*03:01,B*07:02,B*35:03,C*07:02,C*04:01,3140,2998.700
DLBCL10971,A*02:01,A*02:01,B*15:01,B*38:01,C*03:03,C*12:03,2621,2550.233
DLBCL10887,A*02:01,A*02:01,B*15:01,B*15:01,C*03:03,C*03:04,2787,2761.917
DLBCL10507,A*29:02,A*24:03,B*44:03,B*35:08,C*04:01,C*16:01,2579,2462.945


In [723]:
table_mut_a = pd.DataFrame(index=hla_counts_A_combined.index, columns=['count_mut_allele', 'count_wt_allele',
                                                                       'count_mut_notallele', 'count_wt_notallele'])

table_mut_a.insert(0, 'p', np.nan)
table_mut_a.insert(0, 'q', np.nan)

total_a_alleles = len(mutated_a) * 2
total_nota_alleles = 690 * 2 - total_a_alleles

for a in table_mut_a.index:
    mut_a = (hla_types.loc[mutated_a] == a).sum(axis=1).sum()
    mut_nota = total_a_alleles - mut_a
    wt_a = (hla_types.loc[~hla_types.index.isin(mutated_a)] == a).sum(axis=1).sum()
    wt_nota = total_nota_alleles - wt_a
    
    m = np.array([[0, 0]] * 2)
    m[0][0] = mut_a
    m[0][1] = mut_nota
    m[1][0] = wt_a
    m[1][1] = wt_nota
    
    p = fisher_exact_2x2(m, 'two.sided')[0][0]
    
    table_mut_a.loc[a, 'p'] = p
    table_mut_a.loc[a, 'count_mut_allele'] = mut_a
    table_mut_a.loc[a, 'count_wt_allele'] = wt_a
    table_mut_a.loc[a, 'count_mut_notallele'] = mut_nota
    table_mut_a.loc[a, 'count_wt_notallele'] = wt_nota

In [727]:
table_mut_b = pd.DataFrame(index=hla_counts_B_combined.index, columns=['count_mut_allele', 'count_wt_allele',
                                                                       'count_mut_notallele', 'count_wt_notallele'])

table_mut_b.insert(0, 'p', np.nan)
table_mut_b.insert(0, 'q', np.nan)

total_b_alleles = len(mutated_b) * 2
total_notb_alleles = 690 * 2 - total_b_alleles

for b in table_mut_b.index:
    mut_b = (hla_types.loc[mutated_b] == b).sum(axis=1).sum()
    mut_notb = total_b_alleles - mut_b
    wt_b = (hla_types.loc[~hla_types.index.isin(mutated_b)] == b).sum(axis=1).sum()
    wt_notb = total_notb_alleles - wt_b
    
    m = np.array([[0, 0]] * 2)
    m[0][0] = mut_b
    m[0][1] = mut_notb
    m[1][0] = wt_b
    m[1][1] = wt_notb
    
    p = fisher_exact_2x2(m, 'two.sided')[0][0]
    
    table_mut_b.loc[b, 'p'] = p
    table_mut_b.loc[b, 'count_mut_allele'] = mut_b
    table_mut_b.loc[b, 'count_wt_allele'] = wt_b
    table_mut_b.loc[b, 'count_mut_notallele'] = mut_notb
    table_mut_b.loc[b, 'count_wt_notallele'] = wt_notb

In [728]:
table_mut_c = pd.DataFrame(index=hla_counts_C_combined.index, columns=['count_mut_allele', 'count_wt_allele',
                                                                       'count_mut_notallele', 'count_wt_notallele'])

table_mut_c.insert(0, 'p', np.nan)
table_mut_c.insert(0, 'q', np.nan)

total_c_alleles = len(mutated_c) * 2
total_notc_alleles = 690 * 2 - total_c_alleles

for c in table_mut_c.index:
    mut_c = (hla_types.loc[mutated_c] == c).sum(axis=1).sum()
    mut_notc = total_c_alleles - mut_c
    wt_c = (hla_types.loc[~hla_types.index.isin(mutated_c)] == c).sum(axis=1).sum()
    wt_notc = total_notc_alleles - wt_c
    
    m = np.array([[0, 0]] * 2)
    m[0][0] = mut_c
    m[0][1] = mut_notc
    m[1][0] = wt_c
    m[1][1] = wt_notc
    
    p = fisher_exact_2x2(m, 'two.sided')[0][0]
    
    table_mut_c.loc[c, 'p'] = p
    table_mut_c.loc[c, 'count_mut_allele'] = mut_c
    table_mut_c.loc[c, 'count_wt_allele'] = wt_c
    table_mut_c.loc[c, 'count_mut_notallele'] = mut_notc
    table_mut_c.loc[c, 'count_wt_notallele'] = wt_notc

In [729]:
valid_A = hla_counts_A_combined.loc[hla_counts_A_combined['total'] >= 10].index
valid_B = hla_counts_B_combined.loc[hla_counts_B_combined['total'] >= 10].index
valid_C = hla_counts_C_combined.loc[hla_counts_C_combined['total'] >= 10].index

table_mut_a.loc[valid_A, 'q'] = sm.multipletests(table_mut_a.loc[valid_A, 'p'], method='fdr_bh')[1]
table_mut_a = table_mut_a.sort_values(by='q')

table_mut_b.loc[valid_B, 'q'] = sm.multipletests(table_mut_b.loc[valid_B, 'p'], method='fdr_bh')[1]
table_mut_b = table_mut_b.sort_values(by='q')

table_mut_c.loc[valid_C, 'q'] = sm.multipletests(table_mut_c.loc[valid_C, 'p'], method='fdr_bh')[1]
table_mut_c = table_mut_c.sort_values(by='q')

table_mut_a

,q,p,count_mut_allele,count_wt_allele,count_mut_notallele,count_wt_notallele
A*33:03,0.321094,0.035677,4,10,126,1240
A*02:01,0.321094,0.029830,25,355,105,895
A*25:01,0.334729,0.055788,6,24,124,1226
A*03:01,0.339413,0.075425,24,158,106,1092
A*33:01,0.366051,0.128067,4,17,126,1233
A*31:01,0.366051,0.142353,6,30,124,1220
A*24:02,0.366051,0.116411,5,101,125,1149
A*01:01,0.686071,0.381150,17,207,113,1043
A*30:02,0.686071,0.350344,2,11,128,1239
A*11:01,0.686071,0.367040,12,86,118,1164


In [730]:
table_mut_b

,q,p,count_mut_allele,count_wt_allele,count_mut_notallele,count_wt_notallele
B*39:01,0.664766,0.097507,5,16,161,1198
B*07:02,0.664766,0.166192,22,116,144,1098
B*35:02,0.664766,0.136644,3,8,163,1206
B*27:05,0.664766,0.052876,1,42,165,1172
B*40:01,0.664766,0.140742,13,61,153,1153
...,...,...,...,...,...,...
B*54:01,NaN,1.000000,0,4,166,1210
B*56:01,NaN,1.000000,0,5,166,1209
B*57:03,NaN,0.226187,1,1,165,1213
B*58:02,NaN,1.000000,0,1,166,1213


In [731]:
table_mut_c

,q,p,count_mut_allele,count_wt_allele,count_mut_notallele,count_wt_notallele
C*14:02,0.171403,0.009522,4,21,44,1311
C*02:02,0.925622,0.102847,4,48,44,1284
C*16:01,1.000000,1.000000,0,24,48,1308
C*03:03,1.000000,0.514552,1,75,47,1257
C*03:04,1.000000,0.258599,1,104,47,1228
C*06:02,1.000000,0.800945,5,126,43,1206
C*01:02,1.000000,1.000000,1,49,47,1283
C*07:04,1.000000,0.473374,1,17,47,1315
C*08:02,1.000000,0.719365,1,59,47,1273
C*12:03,1.000000,0.185070,5,72,43,1260
